In [174]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure

from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
# import os

### NA 채우기
- 요일+시간가중치+Dacon적용, 누적값 분산한 ch_Test, ch_Train을 각각 50% 이상의 데이터가 존재하는 시간대에서 cut
- Train (X118) 데이터 값이 1개여서 추가 drop
- Arima를 이용해 단순 예측한 24hr을 시간에 맞춰 대입

In [3]:
test = pd.read_csv("data/test.csv", sep = ",")
train = pd.read_csv("data/train.csv", sep =",")

In [4]:
submission = pd.read_csv("data/submission.csv")

In [20]:
changed_train = pd.read_csv("data/train_DHWeight_onDacon.csv", index_col=0, sep=',')
changed_test = pd.read_csv("data/test_DHWeight_onDacon.csv", index_col=0, sep=',')

In [28]:
changed_test.index = pd.to_datetime(changed_test.index) 
changed_train.index = pd.to_datetime(changed_train.index) 

In [47]:
idx = changed_test.count(axis = 'columns') >= 201*0.5
ch_te_TimeCut = changed_test[idx]

In [64]:
arima_test = pd.read_csv("data/TEST_DHWonDacon_24시간예측.csv",index_col=0, sep=',')
arima_train = pd.read_csv("data/TRAIN_DHWonDacon_24시간예측.csv",index_col=0, sep=',')

In [125]:
def fillNA(df1, df2):
    df = pd.DataFrame()
    n=0
    for i in df1.columns:
        n += 1
        li = df1[df1[i].isnull()].index
        tmp = pd.DataFrame(df1[i]) 
        print("%d. current column: %s, NaN count: %d" %(n, i, len(li)))
        for time in li:
            tmp.loc[time] = df2.iloc[h][i]
        df = pd.concat([df,tmp], axis=1) 
    return df

In [126]:
TE_TimeCut_NAfilled = fillNA(ch_te_TimeCut, arima_test)
TE_TimeCut_NAfilled.shape

1. current column: X26, NaN count: 5291
2. current column: X303, NaN count: 46
3. current column: X241, NaN count: 67
4. current column: X435, NaN count: 45
5. current column: X402, NaN count: 50
6. current column: X352, NaN count: 55
7. current column: X305, NaN count: 51
8. current column: X350, NaN count: 41
9. current column: X326, NaN count: 155
10. current column: X299, NaN count: 97
11. current column: X160, NaN count: 163
12. current column: X125, NaN count: 0
13. current column: X211, NaN count: 79
14. current column: X349, NaN count: 50
15. current column: X407, NaN count: 59
16. current column: X330, NaN count: 26
17. current column: X132, NaN count: 71
18. current column: X231, NaN count: 190
19. current column: X225, NaN count: 170
20. current column: X16, NaN count: 5291
21. current column: X286, NaN count: 87
22. current column: X354, NaN count: 71
23. current column: X353, NaN count: 65
24. current column: X76, NaN count: 0
25. current column: X255, NaN count: 90
26. cu

(7016, 200)

In [127]:
idx = changed_train.count(axis = 'columns') >= 1292*0.5
ch_tr_TimeCut = changed_train[idx]

In [128]:
# drop a columns X118 since it is Empty in Arima_Train df
ch_tr_TimeCut = ch_tr_TimeCut.drop(columns = 'X118')

In [129]:
TR_TimeCut_NAfilled = fillNA(ch_tr_TimeCut, arima_train)
TR_TimeCut_NAfilled.shape

1. current column: X692, NaN count: 14
2. current column: X1272, NaN count: 4
3. current column: X553, NaN count: 41
4. current column: X1299, NaN count: 52
5. current column: X598, NaN count: 34
6. current column: X1003, NaN count: 116
7. current column: X1010, NaN count: 16
8. current column: X1216, NaN count: 10
9. current column: X1047, NaN count: 7
10. current column: X381, NaN count: 2
11. current column: X466, NaN count: 33
12. current column: X1420, NaN count: 10
13. current column: X686, NaN count: 2
14. current column: X711, NaN count: 5
15. current column: X1251, NaN count: 6
16. current column: X828, NaN count: 5
17. current column: X1350, NaN count: 39
18. current column: X1066, NaN count: 10
19. current column: X1293, NaN count: 38
20. current column: X1148, NaN count: 33
21. current column: X451, NaN count: 29
22. current column: X174, NaN count: 33
23. current column: X74, NaN count: 2584
24. current column: X626, NaN count: 2
25. current column: X1083, NaN count: 9
26.

206. current column: X1332, NaN count: 37
207. current column: X690, NaN count: 4
208. current column: X1087, NaN count: 9
209. current column: X1213, NaN count: 2
210. current column: X665, NaN count: 2
211. current column: X795, NaN count: 7
212. current column: X430, NaN count: 3
213. current column: X484, NaN count: 37
214. current column: X114, NaN count: 0
215. current column: X1296, NaN count: 75
216. current column: X843, NaN count: 15
217. current column: X280, NaN count: 33
218. current column: X1158, NaN count: 46
219. current column: X1090, NaN count: 4
220. current column: X355, NaN count: 7
221. current column: X290, NaN count: 34
222. current column: X930, NaN count: 251
223. current column: X978, NaN count: 16
224. current column: X961, NaN count: 21
225. current column: X439, NaN count: 5
226. current column: X728, NaN count: 0
227. current column: X103, NaN count: 0
228. current column: X518, NaN count: 6
229. current column: X1108, NaN count: 37
230. current column: 

409. current column: X662, NaN count: 4
410. current column: X415, NaN count: 7
411. current column: X198, NaN count: 33
412. current column: X805, NaN count: 0
413. current column: X360, NaN count: 2
414. current column: X526, NaN count: 2
415. current column: X1154, NaN count: 24
416. current column: X1153, NaN count: 17
417. current column: X956, NaN count: 18
418. current column: X702, NaN count: 6
419. current column: X590, NaN count: 9
420. current column: X1151, NaN count: 54
421. current column: X401, NaN count: 813
422. current column: X36, NaN count: 2748
423. current column: X1239, NaN count: 17
424. current column: X454, NaN count: 8
425. current column: X594, NaN count: 23
426. current column: X681, NaN count: 5
427. current column: X850, NaN count: 4
428. current column: X153, NaN count: 33
429. current column: X1355, NaN count: 43
430. current column: X602, NaN count: 3
431. current column: X781, NaN count: 5
432. current column: X1479, NaN count: 7
433. current column: 

613. current column: X987, NaN count: 7
614. current column: X1019, NaN count: 37
615. current column: X425, NaN count: 1
616. current column: X617, NaN count: 9
617. current column: X1476, NaN count: 43
618. current column: X1496, NaN count: 35
619. current column: X563, NaN count: 23
620. current column: X568, NaN count: 31
621. current column: X958, NaN count: 19
622. current column: X984, NaN count: 25
623. current column: X655, NaN count: 5
624. current column: X1220, NaN count: 3
625. current column: X1042, NaN count: 5
626. current column: X1485, NaN count: 37
627. current column: X610, NaN count: 14
628. current column: X1267, NaN count: 6
629. current column: X735, NaN count: 1
630. current column: X525, NaN count: 3
631. current column: X129, NaN count: 33
632. current column: X1170, NaN count: 60
633. current column: X1405, NaN count: 6
634. current column: X1211, NaN count: 0
635. current column: X812, NaN count: 12
636. current column: X587, NaN count: 6
637. current colum

817. current column: X1105, NaN count: 12
818. current column: X1055, NaN count: 7
819. current column: X1354, NaN count: 32
820. current column: X1058, NaN count: 2
821. current column: X1449, NaN count: 0
822. current column: X147, NaN count: 33
823. current column: X753, NaN count: 6
824. current column: X583, NaN count: 35
825. current column: X1001, NaN count: 28
826. current column: X97, NaN count: 0
827. current column: X1489, NaN count: 25
828. current column: X806, NaN count: 0
829. current column: X1310, NaN count: 50
830. current column: X881, NaN count: 251
831. current column: X1273, NaN count: 11
832. current column: X1398, NaN count: 36
833. current column: X809, NaN count: 4
834. current column: X861, NaN count: 29
835. current column: X1461, NaN count: 6
836. current column: X907, NaN count: 253
837. current column: X188, NaN count: 33
838. current column: X1315, NaN count: 52
839. current column: X394, NaN count: 4
840. current column: X1494, NaN count: 37
841. curren

1016. current column: X1079, NaN count: 36
1017. current column: X1077, NaN count: 12
1018. current column: X1291, NaN count: 68
1019. current column: X794, NaN count: 4
1020. current column: X344, NaN count: 9
1021. current column: X1126, NaN count: 38
1022. current column: X764, NaN count: 4
1023. current column: X33, NaN count: 1430
1024. current column: X1438, NaN count: 3
1025. current column: X714, NaN count: 5
1026. current column: X221, NaN count: 33
1027. current column: X1446, NaN count: 8
1028. current column: X1009, NaN count: 15
1029. current column: X1304, NaN count: 67
1030. current column: X1406, NaN count: 3
1031. current column: X60, NaN count: 1423
1032. current column: X672, NaN count: 5
1033. current column: X832, NaN count: 6
1034. current column: X293, NaN count: 33
1035. current column: X1312, NaN count: 34
1036. current column: X89, NaN count: 0
1037. current column: X1344, NaN count: 68
1038. current column: X1457, NaN count: 4
1039. current column: X191, NaN 

1211. current column: X234, NaN count: 33
1212. current column: X222, NaN count: 33
1213. current column: X1223, NaN count: 6
1214. current column: X1277, NaN count: 1
1215. current column: X980, NaN count: 23
1216. current column: X550, NaN count: 1
1217. current column: X661, NaN count: 5
1218. current column: X84, NaN count: 0
1219. current column: X1176, NaN count: 21
1220. current column: X858, NaN count: 1
1221. current column: X596, NaN count: 9
1222. current column: X1265, NaN count: 2
1223. current column: X1391, NaN count: 36
1224. current column: X715, NaN count: 6
1225. current column: X1419, NaN count: 2
1226. current column: X682, NaN count: 5
1227. current column: X444, NaN count: 8
1228. current column: X1192, NaN count: 17
1229. current column: X328, NaN count: 9
1230. current column: X171, NaN count: 34
1231. current column: X570, NaN count: 26
1232. current column: X720, NaN count: 8
1233. current column: X505, NaN count: 26
1234. current column: X852, NaN count: 11


(3148, 1291)

In [138]:
TE_TimeCut_NAfilled.to_csv("data/TEST_DHweightDaconArima_NAFilled.csv")

In [139]:
TR_TimeCut_NAfilled.to_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv")

### Weather 추가변수
- 나중에 추가

In [190]:
# hourly WEATHER data
w_hr = pd.read_csv("data/weather/인천_시간별__기상자료(16-18)_축소__7월1일.csv",encoding='euc-kr', sep=',')
w_hr2 = w_hr.loc[:,['일시', '기온(°C)', '강수량(mm)', '풍속(m/s)']]
w_hr2['강수량(mm)']=w_hr2.loc[:,'강수량(mm)'].fillna(0)
old = w_hr2.columns
new = ['Time', 'Temp', 'Precipitation', 'Wind']
w_hr2 = w_hr2.rename(columns=dict(zip(old, new)))
w_hr2.Time = pd.to_datetime(w_hr2.Time)
w_hr2.set_index('Time')

,Temp,Precipitation,Wind
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4
2016-07-20 02:00:00,25.4,0.0,0.3
2016-07-20 03:00:00,24.9,0.0,1.1
2016-07-20 04:00:00,24.7,0.0,0.7
2016-07-20 05:00:00,24.7,0.0,1.2
2016-07-20 06:00:00,24.7,0.0,1.1
2016-07-20 07:00:00,24.9,0.0,2.1
2016-07-20 08:00:00,26.1,0.0,3.2


In [191]:
# daily WEATHER data
w_daily = pd.read_csv("data/weather/인천_일별_기상자료(16-18)_축소.csv",encoding='euc-kr', sep=',')
w_daily2 = w_daily.iloc[:,1:5]
old = w_daily2.columns
new = ['Time', 'AvgT', 'MinT', 'MaxT']
w_daily2 = w_daily2.rename(columns=dict(zip(old, new)))
w_daily2.Time = pd.to_datetime(w_daily2.Time)
w_daily2.set_index('Time')

,AvgT,MinT,MaxT
Time,,,
2016-07-20,28.1,24.6,31.6
2016-07-21,28.0,25.0,31.5
2016-07-22,28.5,25.9,32.3
2016-07-23,27.7,25.8,30.2
2016-07-24,27.0,26.4,28.9
2016-07-25,27.0,25.4,29.8
2016-07-26,27.0,25.3,29.2
2016-07-27,26.7,25.9,28.1
2016-07-28,26.2,24.5,28.2


### 파생 및 dummy변수

In [175]:
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [177]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.473264,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342026,1.160255,0.094,0.351,1.109,0.225,0.296678


In [185]:
def dateTimeVar(df):
    df1 = df.copy()
    # trainset, testset의 인덱스를 datetime으로 변환
    if not isinstance(df1.index[1], datetime):
        df1.index = pd.to_datetime(df1.index)
    
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    return df1

In [183]:
test1 = dateTimeVar(test)

In [181]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [ ]:
dummies(test1, ['Mth',''])

### XGBoost Reg